# NOTEBOOK 1: DATA LOADING & CLEANING VALIDATION

# Data Loading and Cleaning

**Purpose:** Load, standardize, and validate the MBIC Media Bias Annotation Dataset

**Dataset Files:**
- `annotators.csv` - Annotator demographic information
- `annotations.xlsx` - Bias judgments by annotators  
- `labeled_dataset.xlsx` - News articles with text content

**Outputs:**
- Standardized UTF-8 CSV files in `data/processed/`
- Ready for exploratory data analysis



In [1]:
# Import libraries
import pandas as pd
import numpy as np

import chardet


In [2]:
# Function to clean special characters in dataframes
def clean_special_characters(df):
    """Clean special characters that cause issues in all string columns"""
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].astype(str).str.replace("’", "'", regex=False)
        df[col] = df[col].str.replace("“", '"', regex=False)
        df[col] = df[col].str.replace("”", '"', regex=False)
        df[col] = df[col].str.replace("–", "-", regex=False)
        df[col] = df[col].str.replace("—", "-", regex=False)
    return df


## Step 1: Detect File Encoding

The `annotators.csv` file may have encoding issues. 


In [3]:
# Create a function to detect the encoding of a CSV file
# This function reads the file in binary mode and uses chardet to detect the encoding
# It returns the encoding and confidence level of the detection
def detect_encoding(file_path):
    """Detect the encoding of a CSV file using chardet"""
    with open(file_path, "rb") as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result


# Detect encoding of annotators.csv
encoding_info = detect_encoding("../data/raw/annotators.csv")
print(f"File encoding: {encoding_info['encoding']}")
print(f"Confidence: {encoding_info['confidence']:.1%}")


File encoding: Windows-1252
Confidence: 73.0%


## Step 2: Standardize CSV File

Convert the annotators.csv file to UTF-8 encoding with comma separators while protecting comma-containing cells.


In [4]:
# Function to standardize CSV files
# This function reads a CSV file with specified separator and encoding,
# performs basic validation, and saves it with UTF-8 encoding and comma separator
# It also prints the shape of the DataFrame and the list of columns
# Additionally, it checks for missing data and prints the total count of missing values
def safe_standardize_csv(
    input_file, output_file, input_sep=";", input_encoding="windows-1252"
):
    """
    Standardize CSV file by converting to UTF-8 encoding and comma separators
    """
    print(f"Standardizing: {input_file}")

    # Load the CSV with detected parameters
    df = pd.read_csv(input_file, sep=input_sep, encoding=input_encoding)

    # Clean special characters
    df = clean_special_characters(df)

    # Basic validation
    print(f"Loaded shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")

    # Check for missing data
    missing_data = df.isnull().sum().sum()
    print(f"Total missing values: {missing_data:,}")

    # Save with proper UTF-8 encoding
    df.to_csv(output_file, encoding="utf-8", index=False)
    print(f"Standardized file saved: {output_file}")

    return df


# Standardize the annotators.csv file
annotators_standardized = safe_standardize_csv(
    "../data/raw/annotators.csv",
    "../data/processed/annotators_utf8.csv",
    input_sep=";",
    input_encoding="Windows-1252",
)


Standardizing: ../data/raw/annotators.csv
Loaded shape: (1345, 9)
Columns: ['id', 'age', 'gender', 'education', 'native_english_speaker', 'political_ideology', 'followed_news_outlets', 'news_check_frequency', 'survey_completed']
Total missing values: 0
Standardized file saved: ../data/processed/annotators_utf8.csv


## Step 3: Convert Excel Files to CSV

Load the Excel files and convert them to UTF-8 CSV format for consistency.


In [5]:
# Load Excel files
annotations = pd.read_excel("../data/raw/annotations.xlsx")
annotations = clean_special_characters(annotations)
print(f"Annotations loaded: {annotations.shape}")

labeled_dataset = pd.read_excel("../data/raw/labeled_dataset.xlsx")
labeled_dataset = clean_special_characters(labeled_dataset)
print(f"Labeled dataset loaded: {labeled_dataset.shape}")

# Convert to UTF-8 CSV files
annotations.to_csv(
    "../data/processed/annotations_utf8.csv", encoding="utf-8", index=False
)
print("annotations.xlsx -> annotations_utf8.csv")

labeled_dataset.to_csv(
    "../data/processed/labeled_dataset_utf8.csv", encoding="utf-8", index=False
)
print("labeled_dataset.xlsx -> labeled_dataset_utf8.csv")


Annotations loaded: (17775, 23)
Labeled dataset loaded: (1700, 12)


/Users/issrar/miniconda3/envs/erdos_summer_2025/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


annotations.xlsx -> annotations_utf8.csv
labeled_dataset.xlsx -> labeled_dataset_utf8.csv


## Step 4: Check Data Structure

Examine the structure of each dataset.


In [6]:
# Check that everything loaded correctly
print(f"Annotators (standardized): {annotators_standardized.shape}")
print(f"Annotations: {annotations.shape}")
print(f"Labeled dataset: {labeled_dataset.shape}")

# Store datasets in dictionary
datasets = {
    "annotators": annotators_standardized,
    "annotations": annotations,
    "labeled_dataset": labeled_dataset,
}

# Analyze each dataset structure
# This will print the shape, columns, and data types of each dataset
print("\nDataset Structures:")
for name, df in datasets.items():
    print(f"\n{name.upper()} DATASET:")
    print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")

    # Show data types
    print("Data types:")
    for col, dtype in df.dtypes.items():
        print(f"  {col}: {dtype}")


Annotators (standardized): (1345, 9)
Annotations: (17775, 23)
Labeled dataset: (1700, 12)

Dataset Structures:

ANNOTATORS DATASET:
Shape: 1,345 rows x 9 columns
Columns: ['id', 'age', 'gender', 'education', 'native_english_speaker', 'political_ideology', 'followed_news_outlets', 'news_check_frequency', 'survey_completed']
Data types:
  id: object
  age: int64
  gender: object
  education: object
  native_english_speaker: object
  political_ideology: int64
  followed_news_outlets: object
  news_check_frequency: object
  survey_completed: object

ANNOTATIONS DATASET:
Shape: 17,775 rows x 23 columns
Columns: ['Unnamed: 0.1', 'Unnamed: 0', 'survey_record_id', 'sentence_id', 'sentence_group_id', 'created_at', 'label', 'words', 'factual', 'group_id', 'text', 'link', 'type', 'topic', 'outlet', 'age', 'gender', 'education', 'native_english_speaker', 'political_ideology', 'followed_news_outlets', 'news_check_frequency', 'survey_completed']
Data types:
  Unnamed: 0.1: int64
  Unnamed: 0: int64


## Step 5: Check for Missing Values

Identify any missing data.


In [7]:
# Check for missing values in each dataset
# This will print the count and percentage of missing values for each column

for name, df in datasets.items():
    print(f"\nMISSING VALUES - {name.upper()}:")

    missing_summary = df.isnull().sum()

    if missing_summary.sum() > 0:
        print("Columns with missing values:")
        for col, missing_count in missing_summary[missing_summary > 0].items():
            missing_pct = (missing_count / len(df)) * 100
            print(f"  {col}: {missing_count:,} ({missing_pct:.1f}%)")
    else:
        print("No missing values detected")



MISSING VALUES - ANNOTATORS:
No missing values detected

MISSING VALUES - ANNOTATIONS:
No missing values detected

MISSING VALUES - LABELED_DATASET:
No missing values detected


## Step 6: Basic Data Cleaning

Not analyzing patterns yet to avoid data leakage.


In [8]:
# Annotations dataset has both judgments and demographics
## Remove unnecessary columns from annotations dataset

# This will drop any unnamed columns and the survey completion column if it exists
clean_annotations = annotations.copy()

# Drop unnamed columns (usually index columns from Excel)
cols_to_drop = [col for col in clean_annotations.columns if "Unnamed" in col]
if cols_to_drop:
    clean_annotations = clean_annotations.drop(columns=cols_to_drop)
    print(f"Dropped columns: {cols_to_drop}")

# Remove survey completion column (redundant because all are completed)
if "survey_completed" in clean_annotations.columns:
    clean_annotations = clean_annotations.drop(columns=["survey_completed"])
    print("Dropped survey_completed column")

print(f"Clean dataset shape: {clean_annotations.shape}")


Dropped columns: ['Unnamed: 0.1', 'Unnamed: 0']
Dropped survey_completed column
Clean dataset shape: (17775, 20)


#### Ordinal encoding: Define ordinal mappings for categorical variables

In [9]:
# Education
education_mapping = {
    "Some high school": 1,
    "High school graduate": 2,
    "Vocational or technical school": 3,
    "Some college": 4,
    "Associate degree": 5,
    "Bachelor's degree": 6,
    "Graduate work": 7,
    "I prefer not to say": np.nan, # will handle below
}
# Apply education mapping
if "education" in clean_annotations.columns:
    clean_annotations["education_ordinal"] = clean_annotations["education"].map(
        education_mapping
    )
    print(f"Education: Created ordinal encoding")


Education: Created ordinal encoding


In [10]:
# News checks frequency
news_checks_mapping = {
    "Never": 1,
    "Very rarely": 2,
    "Several times per month": 3,
    "Several times per week": 4,
    "Every day": 5,
    "Several times per day": 6,
}

# Apply news checks frequency mapping
if "news_check_frequency" in clean_annotations.columns:
    clean_annotations["news_frequency_ordinal"] = clean_annotations[
        "news_check_frequency"
    ].map(news_checks_mapping)
    print(f"News frequency: Created ordinal encoding")


News frequency: Created ordinal encoding


In [11]:
# Native English speaker
native_english_mapping = {
    "Non-native speaker": 1,
    "Near-native speaker": 2,
    "Native speaker": 3,
}

# Apply native English speaker mapping
if "native_english_speaker" in clean_annotations.columns:
    clean_annotations["english_ordinal"] = clean_annotations[
        "native_english_speaker"
    ].map(native_english_mapping)
    print(f"English proficiency: Created ordinal encoding")


English proficiency: Created ordinal encoding


- Create `is_biased` variable based on `label`

In [12]:
clean_annotations["is_biased"] = (clean_annotations["label"] == "Biased").astype(int)

# Verify `label` conversion
bias_counts = clean_annotations["is_biased"].value_counts()
total = len(clean_annotations)


print(
    f"  Not Biased (0): {bias_counts.get(0, 0):,} ({bias_counts.get(0, 0) / total:.1%})"
)
print(f"  Biased (1): {bias_counts.get(1, 0):,} ({bias_counts.get(1, 0) / total:.1%})")


  Not Biased (0): 7,124 (40.1%)
  Biased (1): 10,651 (59.9%)


In [13]:
# Check for any issues in `label` conversion
target_missing = clean_annotations["is_biased"].isnull().sum()
if target_missing > 0:
    print(f"Warning: {target_missing} missing values in converted labels")
else:
    print("No missing values")


No missing values


In [14]:
# Check the new features
print(f"\nNew ordinal features created:")
ordinal_cols = ["education_ordinal", "news_frequency_ordinal", "english_ordinal"]
for col in ordinal_cols:
    if col in clean_annotations.columns:
        print(f"  {col}: {clean_annotations[col].nunique()} unique values")



New ordinal features created:
  education_ordinal: 7 unique values
  news_frequency_ordinal: 6 unique values
  english_ordinal: 3 unique values


In [15]:

# Age: resolve age = 0 issues -> fill with median
if "age" in clean_annotations.columns:
    # Check for age = 0 (invalid ages)
    age_zero_count = (clean_annotations["age"] == 0).sum()

    if age_zero_count > 0:
        # Calculate median from valid ages (excluding 0)
        valid_ages = clean_annotations["age"][clean_annotations["age"] > 0]
        median_age = valid_ages.median()

        # Replace age = 0 with median
        clean_annotations.loc[clean_annotations["age"] == 0, "age"] = median_age

        print(
            f"Age: Replaced {age_zero_count} zero values with median ({median_age:.0f})"
        )
    else:
        print("Age: No zero values found")

    # Show age distribution after fix
    print(
        f"Age range: {clean_annotations['age'].min():.0f} to {clean_annotations['age'].max():.0f}"
    )


Age: Replaced 20 zero values with median (35)
Age range: 18 to 71


In [16]:
# Apply ordinal mappings

# Education 

if "education" in clean_annotations.columns:
    clean_annotations["education_ordinal"] = clean_annotations["education"].map(
        education_mapping
    )
    # Count missing before imputation
    missing_count = clean_annotations["education_ordinal"].isnull().sum()
    median_education = clean_annotations["education_ordinal"].median()
    clean_annotations["education_ordinal"] = clean_annotations[
        "education_ordinal"
    ].fillna(median_education)
    print(
        f"Education: Created ordinal encoding, filled {missing_count} missing with median ({median_education:.0f})"
    )

# News checks frequency
if "news_check_frequency" in clean_annotations.columns:
    clean_annotations["news_frequency_ordinal"] = clean_annotations[
        "news_check_frequency"
    ].map(news_checks_mapping)
    missing_count = clean_annotations["news_frequency_ordinal"].isnull().sum()
    median_news = clean_annotations["news_frequency_ordinal"].median()
    clean_annotations["news_frequency_ordinal"] = clean_annotations[
        "news_frequency_ordinal"
    ].fillna(median_news)
    print(
        f"News frequency: Created ordinal encoding, filled {missing_count} missing with median ({median_news:.0f})"
    )

# Native English speaker
if "native_english_speaker" in clean_annotations.columns:
    clean_annotations["english_ordinal"] = clean_annotations[
        "native_english_speaker"
    ].map(native_english_mapping)
    missing_count = clean_annotations["english_ordinal"].isnull().sum()
    median_english = clean_annotations["english_ordinal"].median()
    clean_annotations["english_ordinal"] = clean_annotations["english_ordinal"].fillna(
        median_english
    )
    print(
        f"English proficiency: Created ordinal encoding, filled {missing_count} missing with median ({median_english:.0f})"
    )


Education: Created ordinal encoding, filled 60 missing with median (6)
News frequency: Created ordinal encoding, filled 0 missing with median (5)
English proficiency: Created ordinal encoding, filled 0 missing with median (3)


In [17]:
# Final check for missing values
remaining_missing = clean_annotations.isnull().sum().sum()
print(f"\nRemaining missing values: {remaining_missing}")



Remaining missing values: 0


# Feature inventory

In [18]:

print("Dataset shape:", clean_annotations.shape)
print("\nActual columns available:")
for i, col in enumerate(clean_annotations.columns, 1):
    print(f"  {i:2d}. {col}")

# Check data types and non-null counts
print(f"\nData types and completeness:")
print(clean_annotations.info())

# Sample of actual data
print(f"\nSample of actual data:")
print(clean_annotations.head(3))


Dataset shape: (17775, 24)

Actual columns available:
   1. survey_record_id
   2. sentence_id
   3. sentence_group_id
   4. created_at
   5. label
   6. words
   7. factual
   8. group_id
   9. text
  10. link
  11. type
  12. topic
  13. outlet
  14. age
  15. gender
  16. education
  17. native_english_speaker
  18. political_ideology
  19. followed_news_outlets
  20. news_check_frequency
  21. education_ordinal
  22. news_frequency_ordinal
  23. english_ordinal
  24. is_biased

Data types and completeness:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17775 entries, 0 to 17774
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   survey_record_id        17775 non-null  object 
 1   sentence_id             17775 non-null  object 
 2   sentence_group_id       17775 non-null  int64  
 3   created_at              17775 non-null  object 
 4   label                   17775 non-null  object 
 5  

In [ ]:
# Safe the cleaned annotations dataset
clean_annotations.to_csv(
    "../data/processed/clean_annotations.csv", encoding="utf-8", index=False
)
print("Cleaned annotations saved to: ../data/processed/clean_annotations.csv")


Cleaned annotations saved to: ../data/processed/clean_annotations.csv


# Final check before moving to next step

In [ ]:
# Final checks before moving to next step
# 1. Check the followed_news_outlets column
print("1. FOLLOWED NEWS OUTLETS COLUMN:")
print("   Sample values:")
print(clean_annotations['followed_news_outlets'].head(10).tolist())
print(f"\n   Data type: {clean_annotations['followed_news_outlets'].dtype}")
print(f"   Unique values: {clean_annotations['followed_news_outlets'].nunique()}")
print(f"   Missing values: {clean_annotations['followed_news_outlets'].isnull().sum()}")
# 2. Check unique values in followed_news_outlets
print("\n2. UNIQUE FOLLOWED NEWS OUTLETS:")
unique_outlets = clean_annotations['followed_news_outlets'].value_counts()
print(f"   Total unique combinations: {len(unique_outlets)}")
print("\n   Top 20 most common combinations:")
print(unique_outlets.head(20))
# 3. Check if outlets are comma-separated or different format
print("\n3. OUTLET FORMAT ANALYSIS:")
sample_outlets = clean_annotations['followed_news_outlets'].dropna().head(20)
print("   Sample outlet entries:")
for i, outlet in enumerate(sample_outlets, 1):
    print(f"   {i:2d}. {outlet}")
# 4. Check for specific outlet names mentioned
print("\n4. SPECIFIC OUTLET ANALYSIS:")
outlet_sample = clean_annotations['followed_news_outlets'].dropna()
print(f"   Total non-null entries: {len(outlet_sample)}")

# Check for common separators
has_comma = outlet_sample.str.contains(',', na=False).sum()
has_semicolon = outlet_sample.str.contains(';', na=False).sum()
has_pipe = outlet_sample.str.contains('|', na=False).sum()

print(f"   Entries with commas: {has_comma}")
print(f"   Entries with semicolons: {has_semicolon}")
print(f"   Entries with pipes: {has_pipe}")
# 5. Check current state of education and other ordinal variables
print("\n5. CURRENT STATE OF ORDINAL VARIABLES:")
print("   Education values:")
print(clean_annotations['education'].value_counts())
print(f"\n   Education ordinal exists: {'education_ordinal' in clean_annotations.columns}")
if 'education_ordinal' in clean_annotations.columns:
    print("   Education ordinal values:")
    print(clean_annotations['education_ordinal'].value_counts().sort_index())
# 6. Check news frequency
print("\n6. NEWS FREQUENCY VALUES:")
print(clean_annotations['news_check_frequency'].value_counts())
print(f"\n   News frequency ordinal exists: {'news_frequency_ordinal' in clean_annotations.columns}")
if 'news_frequency_ordinal' in clean_annotations.columns:
    print("   News frequency ordinal values:")
    print(clean_annotations['news_frequency_ordinal'].value_counts().sort_index())
# 7. Check English proficiency
print("\n7. ENGLISH PROFICIENCY VALUES:")
print(clean_annotations['native_english_speaker'].value_counts())
print(f"\n   English ordinal exists: {'english_ordinal' in clean_annotations.columns}")
if 'english_ordinal' in clean_annotations.columns:
    print("   English ordinal values:")
    print(clean_annotations['english_ordinal'].value_counts().sort_index())
# 8. Check overall dataset state
print("\n8. OVERALL DATASET STATE:")
print(f"   Dataset shape: {clean_annotations.shape}")
print(f"   Columns: {list(clean_annotations.columns)}")
print(f"\n   Missing values by column:")
missing_summary = clean_annotations.isnull().sum()
for col, missing in missing_summary[missing_summary > 0].items():
    print(f"   {col}: {missing}")
# 9. Check for any remaining missing values
print("\n9. REMAINING MISSING VALUES:")
remaining_missing = clean_annotations.isnull().sum().sum()
if remaining_missing > 0:
    print(f"   Total remaining missing values: {remaining_missing}")
else:
    print("   No remaining missing values detected")        


NEWS OUTLET DATA EXPLORATION
1. FOLLOWED NEWS OUTLETS COLUMN:
   Sample values:
["['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']", "['ABC News', 'MSNBC']"]

   Data type: object
   Unique values: 551
   Missing values: 0

2. UNIQUE FOLLOWED NEWS OUTLETS:
   Total unique combinations: 551

   Top 20 most common combinations:
followed_news_outlets
['CNN']                                      1100
['Fox News']                                  980
['New York Times']                            679
['The Washington Post']                       320
['USA Today']                                 300
['ABC News']                                  280
['MSNBC']                                     240
['Fox News', 'CNN']                           220
['New York Times', 'The Washington Post']     180
['Fox News', 'Breitbart']  